In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

''' given a movie ID, retuns a list of the IDs of the five nearest neighbors '''
def back_end(movieName):
    
    ''' loading data base '''
    df = pd.read_csv('https://github.com/ArinB/CA05-kNN/raw/master/movies_recommendation_data.csv')
    
    
    ''' return index of movie by Movie ID'''
    index = df.index[df['Movie Name']==movieName][0]
   

    ''' removing unneeded columns for KNN matrix '''
    to_be_removed=['Movie Name','Movie ID','Label']
    cols=df.columns
    cols=[i for i in cols if i not in to_be_removed]
    X=df[cols]
    
    
    ''' creating knn model, fitting model '''
    knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6, n_jobs=-1)
    knn_model.fit(X)
    
    
    ''' using fit model to return results '''
    distances, indices = knn_model.kneighbors(X.iloc[[index]],n_neighbors=6)
    
    
    ''' strip off self result from indicies '''
    adj_indices = list(indices[0])[1:]
    
    
    ''' return list of movies by Movie Name '''
    neighbors = [df.iloc[i][1] for i in adj_indices]
    
    return neighbors

In [3]:
# pip install fuzzywuzzy
# pip install python-Levenshtein

In [4]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

''' program to browse through the movie database'''
def front_end():
    
    ''' loading data base '''
    df = pd.read_csv('https://github.com/ArinB/CA05-kNN/raw/master/movies_recommendation_data.csv')
    
    ''' print welcome statements'''
    print()
    print()
    print('Welcome to our movie catalog!')
    print()
    print('What kind of movie are you interested in viewing?')
    print('Please enter the genre of movie you would like to see.')
    print()
    
    
    ''' generate list of genre names from dataframe '''
    genres=list(df.columns[3:-1])
    genres.sort()
    print('You can choose between:')
    
    
    ''' print out list of genres and ask for input'''
    for genre in genres:
        print('and {0}'.format(genre) if genre==genres[-1] else '{0}, '.format(genre),end='')
    print()
    genre_choice = input('>>> ')
    
    
    ''' check if genre choice is available and re-ask if necessary '''
    if not genre_choice.title() in genres:
        print('You can choose between:')
        for genre in genres:
            print('and {0}'.format(genre) if genre==genres[-1] else '{0}, '.format(genre),end='')
        genre_choice = input('>>> ')
    print()
    
    
    ''' affirming choice of genre '''
    genre_choice=genre_choice.title()
    print()
    print('You chose {0}. Great choice!'.format(genre_choice))
    print()
    
    
    ''' displaying list of movies within selected genre '''
    print('Here is a list of our {0} movie titles:'.format(genre_choice))
    movies=df[df[genre_choice]==1]['Movie Name'].tolist()
    for movie in movies:
        print(movie)
    print()
    
    ''' get movie choice input'''
    print('Which movie would you like to watch?')
    movie_choice=input(">>> ")
    matching=False
    while not matching:
        for movie in movies:
            
            ''' using fuzzy logic to better match user input to movie name strings '''
            ratio= fuzz.partial_ratio(movie.lower(),movie_choice.lower())
            #print('({0},{1}) ratio: {2}'.format(movie,movie_choice,ratio))
            if ratio > 80:
                matching=True
                final_movie_choice=movie
                
                
            ''' confirm that match was found and re-ask for input if not found '''
        if not matching:
            print('We could not find that title in the list.')
            print()
            print('Here is a list of our {0} movie titles:'.format(genre_choice))
            movies=df[df[genre]==1]['Movie Name'].tolist()
            for movie in movies:
                print(movie)
            print()
            print('Which movie would you like to watch?')
            movie_choice=input(">>> ")
            
            
    ''' affirm choice of movie '''
    print()
    print('Great choice, {}! \nWe do hope you enjoy your cinematic experience.'.format(final_movie_choice))
    print()
    print('Remember if you like {}, be sure to check out following movies:'.format(final_movie_choice))
    
    
    ''' send movie choice to back_end for recommendations and then print returned movie titles'''
    movies_to_watch=back_end(final_movie_choice)
    for title in movies_to_watch:
        print(title)

In [5]:
front_end()



Welcome to our movie catalog!

What kind of movie are you interested in viewing?
Please enter the genre of movie you would like to see.

You can choose between:
Biography, Comedy, Crime, Drama, History, Mystery, and Thriller
>>> biography


You chose Biography. Great choice!

Here is a list of our Biography movie titles:
The Imitation Game
A Beautiful Mind
The Wolf of Wall Street
The Wind Rises
Hacksaw Ridge
12 Years a Slave
Queen of Katwe

Which movie would you like to watch?
>>> wolf of wall

Great choice, The Wolf of Wall Street! 
We do hope you enjoy your cinematic experience.

Remember if you like The Wolf of Wall Street, be sure to check out following movies:
21 Jump Street
Thor: Ragnarok
A Beautiful Mind
Avatar
Spirited Away
